<a href="https://colab.research.google.com/github/srinivasvarma123/Data-Engineering-concepts/blob/main/Complex_struct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark").getOrCreate()

sc = spark.sparkContext
from pyspark.sql.functions import *

In [2]:
data = """
{
    "id": 2,
    "trainer" : "sai",
    "zeyoaddress":{
        "permanentAddress":"hyderabad",
        "temporaryAddress":"chennai"
    }
}
"""


rdd = sc.parallelize([data])
df = spark.read.option("multiline","true").json(rdd)

df.show()
df.printSchema()

flattendf = df.select(
                    "id",
                    "trainer",
                    "zeyoaddress.permanentAddress",
                    "zeyoaddress.temporaryAddress"
)
flattendf.show()
flattendf.printSchema()

+---+-------+--------------------+
| id|trainer|         zeyoaddress|
+---+-------+--------------------+
|  2|    sai|{hyderabad, chennai}|
+---+-------+--------------------+

root
 |-- id: long (nullable = true)
 |-- trainer: string (nullable = true)
 |-- zeyoaddress: struct (nullable = true)
 |    |-- permanentAddress: string (nullable = true)
 |    |-- temporaryAddress: string (nullable = true)

+---+-------+----------------+----------------+
| id|trainer|permanentAddress|temporaryAddress|
+---+-------+----------------+----------------+
|  2|    sai|       hyderabad|         chennai|
+---+-------+----------------+----------------+

root
 |-- id: long (nullable = true)
 |-- trainer: string (nullable = true)
 |-- permanentAddress: string (nullable = true)
 |-- temporaryAddress: string (nullable = true)



In [3]:
data1="""


{
	"place": "Hyderabad",
	"user": {
		"name": "zeyo",
		"address": {
			"number": "40",
			"street": "ashok nagar",
			"pin": "400209"
		}
	}
}


"""

rdd1 = sc.parallelize([data1])
df1 = spark.read.option("multiline", "true").json(rdd1)
df1.show()
df1.printSchema()

flatten_df1 = df1.select(
                    "place",
                    "user.address.number",
                    "user.address.pin",
                    "user.address.street",
                    "user.name"
)
flatten_df1.show()
flatten_df1.printSchema()

+---------+--------------------+
|    place|                user|
+---------+--------------------+
|Hyderabad|{{40, 400209, ash...|
+---------+--------------------+

root
 |-- place: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- number: string (nullable = true)
 |    |    |-- pin: string (nullable = true)
 |    |    |-- street: string (nullable = true)
 |    |-- name: string (nullable = true)

+---------+------+------+-----------+----+
|    place|number|   pin|     street|name|
+---------+------+------+-----------+----+
|Hyderabad|    40|400209|ashok nagar|zeyo|
+---------+------+------+-----------+----+

root
 |-- place: string (nullable = true)
 |-- number: string (nullable = true)
 |-- pin: string (nullable = true)
 |-- street: string (nullable = true)
 |-- name: string (nullable = true)



In [4]:
data2="""
{
    "place": "Hyderabad",
    "user": {
        "name": "zeyo",
        "address": {
            "number": "40",
            "street": "ashok nagar",
            "pin": "400209"
        }
    }
}
"""

rdd2 = sc.parallelize([data2])
df2 = spark.read.option("multivalue", "true").json(rdd2)
df2.show()
df2.printSchema()


flattendf2 = (
            df2.withColumn("number", expr("user.address.number"))
                .withColumn("pin",expr("user.address.pin"))
                .withColumn("street", expr("user.address.street"))
                .withColumn("name", expr("user.name"))
                .drop("user")
)

flattendf2.show()
flattendf2.printSchema()

+---------+--------------------+
|    place|                user|
+---------+--------------------+
|Hyderabad|{{40, 400209, ash...|
+---------+--------------------+

root
 |-- place: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- number: string (nullable = true)
 |    |    |-- pin: string (nullable = true)
 |    |    |-- street: string (nullable = true)
 |    |-- name: string (nullable = true)

+---------+------+------+-----------+----+
|    place|number|   pin|     street|name|
+---------+------+------+-----------+----+
|Hyderabad|    40|400209|ashok nagar|zeyo|
+---------+------+------+-----------+----+

root
 |-- place: string (nullable = true)
 |-- number: string (nullable = true)
 |-- pin: string (nullable = true)
 |-- street: string (nullable = true)
 |-- name: string (nullable = true)

